In [1]:
import numpy as np
from matplotlib import pyplot as plt
import nbimporter
import tkinter as tk
from PIL import Image, ImageDraw, ImageFont
from OptiEngine import FEM_2D, Config, optimize
from FEM_PostProcess import Element, PostProcessor

In [2]:
fixes = [
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [10, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [15, 29, 1, 1],
        [25, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [5, 29, 1, 1],
        [25, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [25, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [15, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [10, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [20, 29, 1, 1]
    ],
    [
        [10, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [29, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [10, 29, 1,1]
    ],
    [
        [0, 29, 1, 1],
        [10, 29, 1, 1]
    ],
    [
        [5, 29, 1, 1],
        [20, 29, 1, 1]
    ],
    [
        [0, 29, 1, 1],
        [15, 29, 1, 1]
    ]
]
loads = [
    [0, 0, 0, -1],
    [20, 0, 0, -1],
    [5, 0, 0, -1],
    [10, 0, 0, -1],
    [0, 0, 0, -1],
    [20, 0, 0, -1],
    [5, 10, 0, -1],
    [15, 0, 0, -1],
    [25, 0, 0, -1],
    [15, 0, 0, -1],
    [29, 0, 0, -1],
    [15, 15, 0, -1],
    [15, 10, 0, -1],
    [5, 5, 0, -1],
    [5, 10, 0, -1],
    [15, 20, 0, -1],
    [25, 5, 0, -1],
    [15, 10, 0, -1],
    [20, 5, 0, -1],
    [18, 0, 0, -1]
]

In [42]:
samples = 20
simp_displs = []
nn_displs = []
dev = 0
for i in range(samples):
    name = f'comapre_{i+1}'
    path = f'C:\\Users\\mihai\\Desktop\\Unik\\4 курс\\Оптимизация\\Результаты\\Compare\\{name}'
    
    nn_density = np.transpose(np.loadtxt(path + '_nn.txt'))
    simp_density = np.transpose(np.loadtxt(path + '_simp.txt'))
    
    mean_ = np.mean(nn_density)
    std_ = np.std(nn_density)
    
    nn_density = (nn_density - mean_) / (std_**2)
    nn_density[nn_density < 0] = 0.0
    nn_density[nn_density > 0.2] = 1.0
    
    sum_ = np.sum(nn_density)/841
    deviation = sum_ / 0.3
    dev += deviation
    
    fix_1_y = fixes[i][0][0]
    fix_1_x = fixes[i][0][1]
    fix_2_y = fixes[i][1][0]
    fix_2_x = fixes[i][1][1]
    load_y = loads[i][0]
    load_x = loads[i][1]
    config = Config()

    config.nx = 30
    config.ny = 30

    config.vol_frac = 0.3
    config.penalize = 3

    config.dense_delta = 0.01
    config.max_iter = 40

    config.E0 = 1
    config.E_void = 1e-9

    config.filter_r = 1.5
    config.move = 0.2

    config.fem_save_iters = [1, 15, 35, 50]
    
    FEM_solver = FEM_2D(config.nx, config.ny, config.filter_r)
    FEM_solver.form_elements()
    
    # SIMP analysis
    FEM_solver.assemble_sparse_K(simp_density.reshape((841,)), config.penalize, config.E_void, config.E0)
    FEM_solver.fix_node(fix_1_y, fix_1_x, True, True)
    FEM_solver.fix_node(fix_2_y, fix_2_x, True, True)
    FEM_solver.apply_load(load_y, load_x, 0, -1)
    displ_simp = FEM_solver.solve_U()
    simp_displs.append(displ_simp)
    FEM_solver.save_txt(f'simp_{i}.txt', simp_density.reshape((841,)))
    
    # NN Analysis
    FEM_solver_2 = FEM_2D(config.nx, config.ny, config.filter_r)
    FEM_solver_2.form_elements()
    
    FEM_solver_2.assemble_sparse_K(nn_density.reshape((841,)), config.penalize, config.E_void, config.E0)
    FEM_solver_2.fix_node(fix_1_y, fix_1_x, True, True)
    FEM_solver_2.fix_node(fix_2_y, fix_2_x, True, True)
    FEM_solver_2.apply_load(load_y, load_x, 0, -1)
    displ_nn =FEM_solver_2.solve_U()
    nn_displs.append(displ_nn)
    FEM_solver_2.save_txt(f'nn_{i}.txt', nn_density.reshape((841,)))

In [43]:
simp_means = []
simp_maxs = []
nn_means = []
nn_maxs = []
for i in range(samples):
    if (i == 10):
        continue
    simp_means.append(np.percentile(np.abs(simp_displs[i]), 25))
    simp_maxs.append(np.max(np.abs(simp_displs[i])))
    nn_means.append(np.percentile(np.abs(nn_displs[i]), 25))
    nn_maxs.append(np.max(np.abs(nn_displs[i])))

In [44]:
ticks = np.arange(19)

In [45]:
simp_means = np.array(simp_means)
nn_means = np.array(nn_means)
simp_maxs = np.array(simp_maxs)
nn_maxs = np.array(nn_maxs)

nn_means[nn_means > 1000] = np.mean(nn_means[nn_means < 1000])
simp_means[simp_means > 1000] = np.mean(simp_means[simp_means < 1000])

In [58]:
ratio = (nn_means/simp_means)
high_perc = np.percentile(ratio, 93)
ratio[ratio > high_perc] = np.mean(ratio[ratio < high_perc])

In [59]:
dev/20

1.1290785992900472

In [60]:
np.mean(ratio)

1.195677844242864